---
title: Analysis
subtitle: The Model Development Process in scikit-learn
authors:
  - name: Ian Carroll
    affiliations:
      - University of Maryland Baltimore County
      - NASA Goddard Space Flight Center
  - name: Rachel Wegener
    affiliations:
      - University of Maryland College Park
github: nasa-sarp/lesson-analysis-i-east
---

::::{grid}

:::{card}
:header: Context 🤔
You've created or have access to great data! Now what? This lesson starts moving from exploration to analysis.
:::

:::{card}
:header: Outcome 🎓
Building a model to make a prediction will become a concrete, well-defined process.
:::

:::{card}
:header: Skills 🤓
A new tool for understanding why your code produces an error, the `%debug` command!
:::

::::

In [ ]:
import holoviews
import hvplot.xarray
import numpy
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import xarray

options = xarray.set_options(display_style='text')
holoviews.opts.defaults(
    holoviews.opts.Curve(active_tools=[], toolbar=None),
    holoviews.opts.Histogram(active_tools=[], toolbar=None),
    holoviews.opts.Scatter(active_tools=[], toolbar=None),
    holoviews.opts.Image(active_tools=[]),
    holoviews.opts.Points(active_tools=[]),
    holoviews.opts.Layout(toolbar=None),
    holoviews.opts.ErrorBars(upper_head=None, lower_head=None)
)

In [ ]:
import scipy.integrate

sol = scipy.integrate.solve_ivp(lambda t, y: (y[1], -9.8), (0, 3), [100, 0], t_eval=numpy.linspace(0, 3, 80))
array = xarray.DataArray(sol.y[0], coords={'time': sol.t}, name='height')
plta = array.hvplot(aspect=1).opts(frame_height=100, aspect=1)
array = xarray.DataArray(
    data=[0.3, 0.45, 0.42, 0.51, 0.48, 0.52, 0.63, 0.65],
    coords={'X': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]},
    name='y'
)
pltb = array.hvplot.scatter(x='X', y='y', aspect=1, frame_height=100) * holoviews.Slope(0.45, .3).opts(color='orange')

In [ ]:
holoviews.extension('bokeh')

## Models: Conceptual, Mathematical, and Numerical

Data analysis is inevitably connected to *some* kind of model, some abstraction that describes how a system behaves. A conceptual model might do that in words or diagrams. A mathematical model uses various sorts of equations. A numerical model ultimately represents the way a system behaves with data. Data analysis is the process of comparing a numerical model to observations: to seeing how well your observations "fit the model".

In some cases, the model is very well defined before you even collect data. Most often a "process model" makes clear numerical predictions before you take your observations into account.

::::{grid} 

:::{card}
Conceptual
: 🌳🍎🤔 Gravity accelerates objects downword according to Newton's second law of motion.
:::

:::{card}
Mathematical
: $$
  \begin{equation}
    \dot{y} &= v \\
    \dot{v} &= -g
  \end{equation}
  $$
:::

:::{card}
Numerical
: {eval}`plta`
:::

::::

In other cases, the model is not well defined *until* you begin comparing it to observations. A statistical model cannot make predictions until it has been trained on some data from the system it is meant to predict.

::::{grid}

:::{card}
Conceptual
: A person's risk of developing diabetes depends on their age, sex, body mass index, blood pressure, and several quantitative blood serum properties.
:::

:::{card}
Mathematical
: $$
  \begin{equation}
    \hat{y} = \alpha + X \beta
  \end{equation}
  $$
:::

:::{card}
Numerical
: {eval}`pltb`
:::

::::

There are different paradigms for the best approach to evaluate the kind of model that requies training. A traditional split in Statistics departments has been between frequentist and Baysian approaches. Both require some solid statistical background, so this lesson instead uses the cross-validation approach that is adopted by most machine-learning analyses.

:::{seealso} Cross-validation
Splitting a dataset into a subset used for training and a distinct subset used for testing makes it possible to evaluate the model fairly. Evaluating a model on the data used to train it will give an overly optimistic assessment.
:::

## A Dataset for Regression

The scikit-learn package builds in a few small datasets that are used in examples throughout the package documentation. In this dataset, the "sample" dimension corresponds to different study participants. The "feature" dimension corresponds to the different variables that may predict disease progression.

In [ ]:
sk_diabetes = sklearn.datasets.load_diabetes()
sk_diabetes.data

In [ ]:
x = xarray.DataArray(
    data=sk_diabetes.data,
    dims=('sample', 'feature'),
    coords={
        'feature': sk_diabetes.feature_names,
    },
    name='x',
)
x

In [ ]:
x.hvplot.hist(groupby=['feature'], widget_location='top')

The response varialbe, what the features may help predict, is a continuous measure of progression towards development of diabetes. The fact that it is continuous, rather than discrete or categorical, is what makes the dataset suitable for regression.

In [ ]:
y = xarray.DataArray(
    data=sk_diabetes.target,
    dims='sample',
    name='y',
)
diabetes = xarray.merge((y, x))
diabetes

::::{grid} 1 1 2 2

:::{seealso} Regression
The outputs, or what a regression model predicts, are continuous: they result from a measurement that could give any number (or any number within some bounds). The inputs to a regression can be continuous or discrete.
:::

:::{seealso} Classification
The outputs, or what a classifier predicts, are discrete and unordered: they result from assigning each observation to one of two or more types. The inputs to a classifier can be continuous or discrete.
:::

::::

Data exploration through visualization can reveal whether any one feature might help predict the response, but we will need a model to explore using all features at the same time.

In [ ]:
diabetes.hvplot.scatter(y='y', x='x', groupby='feature', widget_location='top')

Before using scikit-learn to train and evaluate a model, the dataset must be split for cross-validation. Typically the subsetting is done with randomization, to remove any effect of the order that sample appear in the array.

In [ ]:
train_or_test = numpy.random.choice(
    a=('train', 'test'),
    size=diabetes.sizes['sample'],
    p=(0.8, 0.2),
)
train_or_test

In [ ]:
train = diabetes.where(train_or_test=='train', drop=True)
train

A `%debug` cell lets you dive into the stack of the most recent error and inspect what went wrong. The `train_or_test` array needs to have labeled dimensions for the `where` to work.

In [ ]:
# %debug

In [ ]:
train_or_test = xarray.DataArray(train_or_test, dims='sample')

In [ ]:
train = diabetes.where(train_or_test=='train', drop=True)
train

In [ ]:
test = diabetes.where(train_or_test=='test', drop=True)
test

## Training

The scikit-learn package includes many kinds of models, and does a good job at making the interface to using them all pretty similar. Understanding the case of linear regression opens lots of doors for different models.

In [ ]:
model = sklearn.linear_model.LinearRegression()

:::{seealso} Supervised Learning
Supervised learning is the subset of statistical models that rely on having both inputs and outputs on hand to train. The alternative, "un-supervised" learning involves training models just on inputs; for example, clustering high-dimensional inputs into unkown but similar categories, or change-point detection in a time series.
:::

In [ ]:
x = train['x']
y = train['y']

Every scikit-learn model for supervised learning has a `fit` method that takes the inputs (a.k.a. features or predictors) as the first argument and the outputs (a.k.a. targets, labels, or responses) as the second argument.

In [ ]:
model = model.fit(x, y)

A trained model can now be used to make predictions based on the inputs alone. Note that "prediction" is not used in modeling to mean predicting into the future (which is "forecasting"). The name "estimate" might be more logical.

In [ ]:
train['estimate'] = ('sample', model.predict(x))
train

If the fitting procedure has worked, then the estimate should track the outputs "y". For the case of univariate outputs, a one:one plot provides a good visual check.

In [ ]:
(
    train.drop_dims('feature').hvplot.scatter(x='y', y='estimate')
    * holoviews.Slope(1, 0).opts(color='orange')
)

:::{attention} Exercise 1
Open the "practice.ipynb" notebook and begin the first exercise.
:::

## Evaluation

Cross-validation requires that any model evaluation should be performed on data that was not used during the fitting process.

In [ ]:
x = test['x']
y = test['y']
test['estimate'] = ('sample', model.predict(x))
test = test.drop_dims('feature')

In [ ]:
(
    test.hvplot.scatter(x='y', y='estimate')
    * holoviews.Slope(1, 0).opts(color='orange')
)

Quantitative metrics of the quality of the model begins with examining the residuals, of the difference between the observations and the estimates.

In [ ]:
test['residual'] = test['y'] - test['estimate']
test['cludge'] = test['y'] * 0 # sorry about this, but it gets errorbars in the plot below

In [ ]:
(
    test.hvplot.scatter(x='y', y='estimate')
    * test.hvplot.errorbars(x='y', y='estimate', yerr1='cludge', yerr2='residual', hover=[])
    * holoviews.Slope(1, 0).opts(color='orange')
)

Analysis of the residuals beings simply, but can grow fairly sophisticated. One overall summary is called the coefficient of determination, or $R^2$.

In [ ]:
RSS = (test['residual'] ** 2).sum()
TSS = test['y'].var() * test.sizes['sample']
1 - RSS / TSS

This or some other scalar metric is so useful, especially for checking for overfitting, that most scikit-learn models have a built in method for returning this model "score".

:::{seealso} Overfitting
When a model learns to predict the training data too well, it sometimes starts to perform worse on the testing data. The simplest way to cause overfitting is to train on too small a data sample.
:::

When the score on the test data is worse than the score on the training data, that means the model does not generalize. Unfortunately, there is not any particular difference between the train and test score that is decidedly too big. There is always noise, but this one looks okay!

In [ ]:
model.score(x, y)

In [ ]:
model.score(train['x'], train['y'])

Beyond the overall score, and once overfitting has been ruled out, the next step in examining the model residuals is to consider their histogram. Residuals are ideally just noise, meaning the histogram should look like the histogram of a random variable. Different models allow different choices of the random variable, but without going into details, this histogram looks okay!

In [ ]:
(
    test['residual'].hvplot.hist()
    * holoviews.VLine(test['residual'].mean().item()).opts(color='red', line_dash='dotted')
    * holoviews.VLine(test['residual'].median().item())
)

Even if the distribution looks random, closer examination could reveal patterns in the residuals. For this model, the residuals appear to correlate with the response.

In [ ]:
test.hvplot.scatter(x='y', y='residual', groupby=[])

The correlation means there will be some bias in the estimate: a truly high risk of diabetes is underestimated (the estimate is lower than the observation). That's not a good model to use in a health care setting, so it is time to start considering a more flexible model. But beware! With great model flexibility, comes great potential for overfitting.

:::{attention} Exercise 2
Open the "practice.ipynb" notebook and tackle the second exercise.
:::

## Next Steps

- Feature importance

- Multiple ways to build and evaluate models found in different packages
  - Frequentist: statsmodels
  - Bayesian: PyMC
  - Machine Learning: scikit-learn, tensorflow, PyTorch

### Closing Poll

The [closing poll](https://PollEv.com/clickable_images/WHersUlQe5SsG68KOHyfL/respond) which is, as all the others are, anonymous.

:::{danger} Shutdown
Please shut down your server! (File > Hub Control Panel)
:::